In [1]:
import time

from src.swagger import SwaggerParser
from src.config import Config
import json
import os

os.environ["model"] = "gpt-4o-2024-05-13"
os.environ["language_model_key"] = ""

# root_path = "/Users/naariah/Documents/Python_Codes/api-suts/specifications/v3/languageTool.json"
root_path = "/Users/naariah/Documents/Python_Codes/api-suts/specifications/v3/gitlab-groups-13.json"
config = Config()
config.swagger = root_path
parser = SwaggerParser(config)
operations = parser.extract()

2024-11-06 19:33:30.723 | DEBUG    | src.swagger:__init__:28 - Parsing specification file


In [2]:
operations

[get:/groups,
 post:/groups,
 get:/groups/{id},
 put:/groups/{id},
 delete:/groups/{id},
 post:/groups/{id}/share,
 delete:/groups/{id}/share/{group_id},
 get:/groups/{id}/subgroups,
 get:/groups/{id}/hooks,
 post:/groups/{id}/hooks,
 get:/groups/{id}/hooks/{hook_id},
 put:/groups/{id}/hooks/{hook_id},
 delete:/groups/{id}/hooks/{hook_id},
 get:/groups/{id}/descendant_groups,
 get:/groups/{id}/projects,
 post:/groups/{id}/projects/{project_id},
 get:/groups/{id}/projects/shared]

In [ ]:
from collections import defaultdict


def create_resource_node(operations):
    def merge_resource(resource, resource_tree):
        for parent, children in resource_tree.items():
            for child in children.copy():
                if child + "s" in children:
                    children.remove(child)
                    if child + "s" in resource:
                        resource[child + "s"].extend(resource[child])
                        del resource[child]

    resource = {}
    resource_tree = defaultdict(set)

    for operation in sorted(operations, key=lambda x: len(x.path.__repr__().split("/"))):
        path = operation.path.__repr__()
        split_path = path.split("/")
        if "{" in split_path[-1]:
            resource_name = split_path[-2]
        else:
            resource_name = split_path[-1]
        if resource_name not in resource:
            resource[resource_name] = []
        resource[resource_name].append(operation.__repr__())

        for previous_resource in resource.keys():
            if previous_resource in split_path and previous_resource != resource_name:
                resource_tree[previous_resource].add(resource_name)

    merge_resource(resource, resource_tree)

    return resource, resource_tree


resource, resource_tree = create_resource_node(operations)
resource, resource_tree

In [ ]:
from src.rest import PathParam

info = []
for operation in operations:
    info.append(
        {
            "name": operation.__repr__(),
            "description": operation.description,
        }
    )

In [ ]:
info

In [ ]:
operation = operations[3]
info = []
for f in operation.get_leaf_factors():
    info.append(
        {
            "name": f.get_global_name,
            "type": f.type.value,
            "description": f.description,
        }
    )

In [ ]:
from openai import OpenAI
from src.template import *
from src.languagemodel.llm import count_tokens

client = OpenAI(api_key=os.environ["language_model_key"], base_url="https://api.bianxieai.com/v1")
messages = []
messages.append({"role": "system", "content": SystemRole.SYS_ROLE_SEQUENCE})
prompt = INFO.SEQUENCE.format(info) + Task.SEQUENCE
messages.append({"role": "user", "content": prompt})

In [ ]:
messages

In [ ]:
count_tokens(messages, os.environ["model"])

In [ ]:
from loguru import logger

while True:
    try:
        start = time.time()
        response = client.chat.completions.create(
            model=os.environ["model"],
            messages=messages,
            temperature=0.5,
            top_p=0.99,
            frequency_penalty=0,
            presence_penalty=0,
            max_tokens=4096,
            response_format={"type": "json_object"}
        )
        end = time.time()
        print("Time taken: ", end - start)
        break
    except Exception as e:
        logger.error(f"Error in calling language model: {e}")
        logger.error("Retrying...")

In [ ]:
json.loads(response.choices[0].message.content)

In [ ]:
import rstr


def generate_random_string_from_pattern(pattern, min_length=0, max_length=None):
    try:
        generated_str = rstr.xeger(pattern)
        if max_length:
            generated_str = generated_str[min_length:max_length - 1]
        return generated_str
    except Exception:
        pass
    return None


pattern = r'[A-Za-z0-9!@#$%^&*()_+]{10,15}'
random_string = generate_random_string_from_pattern(pattern)
print(random_string)

In [11]:
import os
import time
import json
from openai import OpenAI

os.environ["model"] = "gpt-4o-2024-05-13"
os.environ["language_model_key"] = "sk-QKBW0aRWkwGm7anAD0Bb7d469e6e4d40B6E3B7B4D4589a61"


def call(client, model, messages):
    return client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0.5,
        top_p=0.99,
        frequency_penalty=0,
        presence_penalty=0,
        max_tokens=4096,
        response_format={"type": "json_object"}
    )

In [12]:
prompt1 = """
Given the operation and its parameters, identify all prerequisite 2 schemas for retrieving information related to the operation’s parameters. 
Below is the operation and its parameters: 
post-/booking: 
    flightId: integer 
    departureDate: string 
    arrivalDate: string
    
Below is the list of all schemas and their properties: 
schemas: 
    Flight: 
        type: object 
        properties: 
            id: 
                type: integer 
            origin: 
                type: string 
            destination: 
                type: string 
    Booking: 
        type: object 
        properties: 
        flight: 
            $ref: ’#/components/schemas/Flight’ 
        departureDate:
            type: string 
            format: date 
        arrivalDate: 
            type: string 
            format: date
        passengerName:
            type: string
        passengerAge:
            type: integer
            
Please format the prerequisite schemas as json object, the formate is {operation: {schema: {parameter of the operation: the parameter of the schema that is required}}}.
"""

bak1 = """
Please format the prerequisite schemas as json object, the formate is {operation: {schema: {parameter of the operation: the parameter of the schema that is required}}}.
Note that not all the parameters of the operation has the dependency on the schema.
"""
bak2 = """
Format your answer in JSON object.
Please format the prerequisite schemas in the following structure: <parameter of the operation> -> <equivalent operation of the relevant schema>
"""

prompt2 = """
Given the schema and its properties in the OpenAPI specification of an API application, your task is to identify the prerequisite schemas that need to be created before establishing the mentioned schema.
Below is the schema and its properties
Booking: 
        type: object 
        properties: 
        flight: 
            $ref: ’#/components/schemas/Flight’ 
        departureDate:
            type: string 
            format: date 
        arrivalDate: 
            type: string 
            format: date
        passengerName:
            type: string
        passengerAge:
            type: integer
            
Below is the list of all schemas and their properties:
schemas: 
    Flight: 
        type: object 
        properties: 
            id: 
                type: integer 
            origin: 
                type: string 
            destination: 
                type: string 
    Booking: 
        type: object 
        properties: 
        flight: 
            $ref: ’#/components/schemas/Flight’ 
        departureDate:
            type: string 
            format: date 
        arrivalDate: 
            type: string 
            format: date
        passengerName:
            type: string
        passengerAge:
            type: integer
Return in separated lines. No explanation needed.
Format your answer in JSON object.
"""

In [13]:
client = OpenAI(api_key=os.environ["language_model_key"], base_url="https://api.bianxie.ai/v1")
model1= "gpt-4o-2024-05-13"
model2 = "gpt-3.5-turbo"
for i in range(1,6):
    print(f"***************************************************Round {i}***************************************************")
    print("OS GPT3.5")
    messages = [{"role": "user", "content": prompt1}]
    response = call(client, model2,messages)
    print(json.loads(response.choices[0].message.content))
    print("OS GPT4")
    messages = [{"role": "user", "content": prompt1}]
    response = call(client, model1,messages)
    print(json.loads(response.choices[0].message.content))
    print("SS GPT3.5")
    messages = [{"role": "user", "content": prompt2}]
    response = call(client, model2,messages)
    print(json.loads(response.choices[0].message.content))
    print("SS GPT4")
    messages = [{"role": "user", "content": prompt2}]
    response = call(client, model1,messages)
    print(json.loads(response.choices[0].message.content))

***************************************************Round 1***************************************************
OS GPT3.5
{'post-/booking': {'Flight': {'flightId': 'id'}, 'Booking': {'departureDate': 'departureDate', 'arrivalDate': 'arrivalDate', 'flight': 'flight'}}}
OS GPT4
{'post-/booking': {'Flight': {'flightId': 'id'}, 'Booking': {'departureDate': 'departureDate', 'arrivalDate': 'arrivalDate'}}}
SS GPT3.5
{'prerequisite_schemas': {'Flight': {'type': 'object', 'properties': {'id': {'type': 'integer'}, 'origin': {'type': 'string'}, 'destination': {'type': 'string'}}}}}
SS GPT4
{'prerequisiteSchemas': ['Flight']}
***************************************************Round 2***************************************************
OS GPT3.5
{'post-/booking': {'Booking': {'flight': 'id', 'departureDate': 'departureDate', 'arrivalDate': 'arrivalDate'}}}
OS GPT4
{'post-/booking': {'Flight': {'flightId': 'id'}, 'Booking': {'departureDate': 'departureDate', 'arrivalDate': 'arrivalDate'}}}
SS GPT3.5
{